<a href="https://www.kaggle.com/code/aayushsin7a/algerian-forest-fires-eda-fe-regression?scriptVersionId=157718972" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Import the required library
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import pandas as pd 
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline
import seaborn as sns
import folium

import warnings
warnings.filterwarnings("ignore")

print("All the libraries loaded successfully")

/kaggle/input/algerian-forest-fires-dataset/Algerian_forest_fires_dataset.csv
All the libraries loaded successfully


> The dataset includes 244 instances that regroup a data of two regions of Algeria, namely the Bejaia region located in the northeast of Algeria and the Sidi Bel-abbes region located in the northwest of Algeria.

- 122 instances for each region.

- The period from June 2012 to September 2012.
- The dataset includes 11 attributes and 1 output attribute (class)
- The 244 instances have been classified into 'fire' (138 classes) and 'not fire' (106 classes) classes.

> Attribute Information:

- 1. Date : (DD/MM/YYYY) Day, month ('june' to 'september'), year (2012)
- 2. Weather data observations 2. Temp : temperature noon (temperature max) in Celsius degrees: 22 to 42 
- 3. RH : Relative Humidity in %: 21 to 90 
- 4. Ws :Wind speed in km/h: 6 to 29 
- 5. Rain: total day in mm: 0 to 16.8 FWI Components 
- 6. Fine Fuel Moisture Code (FFMC) index from the FWI system: 28.6 to 92.5 
- 7. Duff Moisture Code (DMC) index from the FWI system: 1.1 to 65.9 
- 8. Drought Code (DC) index from the FWI system: 7 to 220.4 
- 9. Initial Spread Index (ISI) index from the FWI system: 0 to 18.5 
- 10. Buildup Index (BUI) index from the FWI system: 1.1 to 68 
- 11. Fire Weather Index (FWI) Index: 0 to 31.1 
- 12. Classes: two classes, namely Fire and not Fire

In [2]:
# read the dataset 
df = pd.read_csv("/kaggle/input/algerian-forest-fires-dataset/Algerian_forest_fires_dataset.csv")

In [3]:
df.head(3)

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
0,1,6,2012,29,57,18,0,65.7,3.4,7.6,1.3,3.4,0.5,not fire
1,2,6,2012,29,61,13,1.3,64.4,4.1,7.6,1,3.9,0.4,not fire
2,3,6,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,not fire


In [4]:
df.shape

(247, 14)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   day          246 non-null    object
 1   month        245 non-null    object
 2   year         245 non-null    object
 3   Temperature  245 non-null    object
 4    RH          245 non-null    object
 5    Ws          245 non-null    object
 6   Rain         245 non-null    object
 7   FFMC         245 non-null    object
 8   DMC          245 non-null    object
 9   DC           245 non-null    object
 10  ISI          245 non-null    object
 11  BUI          245 non-null    object
 12  FWI          245 non-null    object
 13  Classes      244 non-null    object
dtypes: object(14)
memory usage: 27.1+ KB


# Data Cleaning 

In [6]:
df.isnull().sum()

day            1
month          2
year           2
Temperature    2
 RH            2
 Ws            2
Rain           2
FFMC           2
DMC            2
DC             2
ISI            2
BUI            2
FWI            2
Classes        3
dtype: int64

In [7]:
df[df.isnull().any(axis=1)]

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,Sidi-Bel Abbes Region Dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,14,7,2012,37,37,18,0.2,88.9,12.9,14.6 9,12.5,10.4,fire,NaN


> ### Diagnose our dataframe 

In [8]:
def diagnose_dataframe(df, verbose=True):
  """
  Performs various initial inspections on a pandas dataframe and provides details.

  Args:
    df: The pandas dataframe to diagnose.
    verbose: Whether to print details for each inspection.

  Returns:
    A dictionary containing the results of the inspections.
  """

  results = {}

  # Basic information
  results["shape"] = df.shape
  results["columns"] = list(df.columns)
  results["dtypes"] = df.dtypes.to_dict()
  results["null_counts"] = df.isnull().sum().to_dict()

  # Descriptive statistics
  results["describe"] = df.describe(include='all')

  # Unique values and counts
  for col in df.columns:
    results[f"unique_values_{col}"] = df[col].unique()
    results[f"value_counts_{col}"] = df[col].value_counts().to_dict()

  # Missing value analysis
  missing_values_pct = (df.isnull().sum() / df.shape[0]) * 100
  results["missing_values_pct"] = missing_values_pct.to_dict()
  missing_values_threshold = 0.5  # Set a threshold for high missing value percentage
  high_missing_values = missing_values_pct[missing_values_pct > missing_values_threshold]
  results["high_missing_values"] = list(high_missing_values.index)

  # Duplicate rows
  duplicate_rows = df[df.duplicated()]
  results["duplicate_rows_count"] = len(duplicate_rows)

  # Outlier analysis
  # ... (implement your own outlier analysis strategy)

  if verbose:
    for key, value in results.items():
      print(f"\n{key}:\n{value}")

  return results

diagnose_dataframe(df)


shape:
(247, 14)

columns:
['day', 'month', 'year', 'Temperature', ' RH', ' Ws', 'Rain ', 'FFMC', 'DMC', 'DC', 'ISI', 'BUI', 'FWI', 'Classes  ']

dtypes:
{'day': dtype('O'), 'month': dtype('O'), 'year': dtype('O'), 'Temperature': dtype('O'), ' RH': dtype('O'), ' Ws': dtype('O'), 'Rain ': dtype('O'), 'FFMC': dtype('O'), 'DMC': dtype('O'), 'DC': dtype('O'), 'ISI': dtype('O'), 'BUI': dtype('O'), 'FWI': dtype('O'), 'Classes  ': dtype('O')}

null_counts:
{'day': 1, 'month': 2, 'year': 2, 'Temperature': 2, ' RH': 2, ' Ws': 2, 'Rain ': 2, 'FFMC': 2, 'DMC': 2, 'DC': 2, 'ISI': 2, 'BUI': 2, 'FWI': 2, 'Classes  ': 3}

describe:
        day month  year Temperature   RH   Ws Rain   FFMC  DMC   DC  ISI  BUI  \
count   246   245   245         245  245  245   245   245  245  245  245  245   
unique   33     5     2          20   63   19    40   174  167  199  107  175   
top       1     7  2012          35   64   14     0  88.9  7.9    8  1.1    3   
freq      8    62   244          29   10   43   13

{'shape': (247, 14),
 'columns': ['day',
  'month',
  'year',
  'Temperature',
  ' RH',
  ' Ws',
  'Rain ',
  'FFMC',
  'DMC',
  'DC',
  'ISI',
  'BUI',
  'FWI',
  'Classes  '],
 'dtypes': {'day': dtype('O'),
  'month': dtype('O'),
  'year': dtype('O'),
  'Temperature': dtype('O'),
  ' RH': dtype('O'),
  ' Ws': dtype('O'),
  'Rain ': dtype('O'),
  'FFMC': dtype('O'),
  'DMC': dtype('O'),
  'DC': dtype('O'),
  'ISI': dtype('O'),
  'BUI': dtype('O'),
  'FWI': dtype('O'),
  'Classes  ': dtype('O')},
 'null_counts': {'day': 1,
  'month': 2,
  'year': 2,
  'Temperature': 2,
  ' RH': 2,
  ' Ws': 2,
  'Rain ': 2,
  'FFMC': 2,
  'DMC': 2,
  'DC': 2,
  'ISI': 2,
  'BUI': 2,
  'FWI': 2,
  'Classes  ': 3},
 'describe':         day month  year Temperature   RH   Ws Rain   FFMC  DMC   DC  ISI  BUI  \
 count   246   245   245         245  245  245   245   245  245  245  245  245   
 unique   33     5     2          20   63   19    40   174  167  199  107  175   
 top       1     7  2012          35 

In [9]:
df.drop([122,123],axis=0,inplace=True)

In [10]:
df[df.isnull().any(axis=1)]

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
168,14,7,2012,37,37,18,0.2,88.9,12.9,14.6 9,12.5,10.4,fire,NaN


In [11]:
df.loc[:122,"Region"]=0
df.loc[122:,"Region"]=1

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245 entries, 0 to 246
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   day          245 non-null    object 
 1   month        245 non-null    object 
 2   year         245 non-null    object 
 3   Temperature  245 non-null    object 
 4    RH          245 non-null    object 
 5    Ws          245 non-null    object 
 6   Rain         245 non-null    object 
 7   FFMC         245 non-null    object 
 8   DMC          245 non-null    object 
 9   DC           245 non-null    object 
 10  ISI          245 non-null    object 
 11  BUI          245 non-null    object 
 12  FWI          245 non-null    object 
 13  Classes      244 non-null    object 
 14  Region       245 non-null    float64
dtypes: float64(1), object(14)
memory usage: 38.7+ KB


In [13]:
df['Region'] = df['Region'].astype('int')

In [14]:
df.head()

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes,Region
0,1,6,2012,29,57,18,0,65.7,3.4,7.6,1.3,3.4,0.5,not fire,0
1,2,6,2012,29,61,13,1.3,64.4,4.1,7.6,1,3.9,0.4,not fire,0
2,3,6,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,not fire,0
3,4,6,2012,25,89,13,2.5,28.6,1.3,6.9,0,1.7,0,not fire,0
4,5,6,2012,27,77,16,0,64.8,3,14.2,1.2,3.9,0.5,not fire,0


In [15]:
df = df.dropna().reset_index(drop=True)

In [16]:
df.isnull().sum()

day            0
month          0
year           0
Temperature    0
 RH            0
 Ws            0
Rain           0
FFMC           0
DMC            0
DC             0
ISI            0
BUI            0
FWI            0
Classes        0
Region         0
dtype: int64

In [17]:
df.iloc[[122]]

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes,Region
122,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes,1


In [18]:
df=df.drop(122).reset_index(drop=True)

In [19]:
df.iloc[[122]]

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes,Region
122,1,6,2012,32,71,12,0.7,57.1,2.5,8.2,0.6,2.8,0.2,not fire,1


In [20]:
df.columns

Index(['day', 'month', 'year', 'Temperature', ' RH', ' Ws', 'Rain ', 'FFMC',
       'DMC', 'DC', 'ISI', 'BUI', 'FWI', 'Classes  ', 'Region'],
      dtype='object')

In [21]:
# Remove spaces from column names
df.columns = df.columns.str.strip()

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   day          243 non-null    object
 1   month        243 non-null    object
 2   year         243 non-null    object
 3   Temperature  243 non-null    object
 4   RH           243 non-null    object
 5   Ws           243 non-null    object
 6   Rain         243 non-null    object
 7   FFMC         243 non-null    object
 8   DMC          243 non-null    object
 9   DC           243 non-null    object
 10  ISI          243 non-null    object
 11  BUI          243 non-null    object
 12  FWI          243 non-null    object
 13  Classes      243 non-null    object
 14  Region       243 non-null    int64 
dtypes: int64(1), object(14)
memory usage: 28.6+ KB


In [23]:
# Change data types (required cols as integers, Month,year,temperature,rh,ws)
df[['month','day','year','Temperature','RH','Ws']] = df[['month','day','year','Temperature','RH','Ws']].astype('int')

In [24]:
# Changing the other columns to float dataframe 

In [25]:
[features for features in df.columns if df[features].dtypes=='O']

['Rain', 'FFMC', 'DMC', 'DC', 'ISI', 'BUI', 'FWI', 'Classes']

In [26]:
objects=[features for features in df.columns if df[features].dtypes=='O']

In [27]:
objects

['Rain', 'FFMC', 'DMC', 'DC', 'ISI', 'BUI', 'FWI', 'Classes']

In [28]:
for i in objects:
    if i !='Classes':
        df[i]=df[i].astype('float')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   day          243 non-null    int64  
 1   month        243 non-null    int64  
 2   year         243 non-null    int64  
 3   Temperature  243 non-null    int64  
 4   RH           243 non-null    int64  
 5   Ws           243 non-null    int64  
 6   Rain         243 non-null    float64
 7   FFMC         243 non-null    float64
 8   DMC          243 non-null    float64
 9   DC           243 non-null    float64
 10  ISI          243 non-null    float64
 11  BUI          243 non-null    float64
 12  FWI          243 non-null    float64
 13  Classes      243 non-null    object 
 14  Region       243 non-null    int64  
dtypes: float64(7), int64(7), object(1)
memory usage: 28.6+ KB


# Diagnose the cleaned dataframe 

In [30]:
def diagnose_dataframe(df, verbose=True):
  """
  Performs various initial inspections on a pandas dataframe and provides details.

  Args:
    df: The pandas dataframe to diagnose.
    verbose: Whether to print details for each inspection.

  Returns:
    A dictionary containing the results of the inspections.
  """

  results = {}

  # Basic information
  results["shape"] = df.shape
  results["columns"] = list(df.columns)
  results["dtypes"] = df.dtypes.to_dict()
  results["null_counts"] = df.isnull().sum().to_dict()

  # Descriptive statistics
  results["describe"] = df.describe(include='all')

  # Unique values and counts
  for col in df.columns:
    results[f"unique_values_{col}"] = df[col].unique()
    results[f"value_counts_{col}"] = df[col].value_counts().to_dict()

  # Missing value analysis
  missing_values_pct = (df.isnull().sum() / df.shape[0]) * 100
  results["missing_values_pct"] = missing_values_pct.to_dict()
  missing_values_threshold = 0.5  # Set a threshold for high missing value percentage
  high_missing_values = missing_values_pct[missing_values_pct > missing_values_threshold]
  results["high_missing_values"] = list(high_missing_values.index)

  # Duplicate rows
  duplicate_rows = df[df.duplicated()]
  results["duplicate_rows_count"] = len(duplicate_rows)

  # Outlier analysis
  # ... (implement your own outlier analysis strategy)

  if verbose:
    for key, value in results.items():
      print(f"\n{key}:\n{value}")

  return results

In [31]:
diagnose_dataframe(df)


shape:
(243, 15)

columns:
['day', 'month', 'year', 'Temperature', 'RH', 'Ws', 'Rain', 'FFMC', 'DMC', 'DC', 'ISI', 'BUI', 'FWI', 'Classes', 'Region']

dtypes:
{'day': dtype('int64'), 'month': dtype('int64'), 'year': dtype('int64'), 'Temperature': dtype('int64'), 'RH': dtype('int64'), 'Ws': dtype('int64'), 'Rain': dtype('float64'), 'FFMC': dtype('float64'), 'DMC': dtype('float64'), 'DC': dtype('float64'), 'ISI': dtype('float64'), 'BUI': dtype('float64'), 'FWI': dtype('float64'), 'Classes': dtype('O'), 'Region': dtype('int64')}

null_counts:
{'day': 0, 'month': 0, 'year': 0, 'Temperature': 0, 'RH': 0, 'Ws': 0, 'Rain': 0, 'FFMC': 0, 'DMC': 0, 'DC': 0, 'ISI': 0, 'BUI': 0, 'FWI': 0, 'Classes': 0, 'Region': 0}

describe:
               day       month    year  Temperature          RH          Ws  \
count   243.000000  243.000000   243.0   243.000000  243.000000  243.000000   
unique         NaN         NaN     NaN          NaN         NaN         NaN   
top            NaN         NaN     Na

{'shape': (243, 15),
 'columns': ['day',
  'month',
  'year',
  'Temperature',
  'RH',
  'Ws',
  'Rain',
  'FFMC',
  'DMC',
  'DC',
  'ISI',
  'BUI',
  'FWI',
  'Classes',
  'Region'],
 'dtypes': {'day': dtype('int64'),
  'month': dtype('int64'),
  'year': dtype('int64'),
  'Temperature': dtype('int64'),
  'RH': dtype('int64'),
  'Ws': dtype('int64'),
  'Rain': dtype('float64'),
  'FFMC': dtype('float64'),
  'DMC': dtype('float64'),
  'DC': dtype('float64'),
  'ISI': dtype('float64'),
  'BUI': dtype('float64'),
  'FWI': dtype('float64'),
  'Classes': dtype('O'),
  'Region': dtype('int64')},
 'null_counts': {'day': 0,
  'month': 0,
  'year': 0,
  'Temperature': 0,
  'RH': 0,
  'Ws': 0,
  'Rain': 0,
  'FFMC': 0,
  'DMC': 0,
  'DC': 0,
  'ISI': 0,
  'BUI': 0,
  'FWI': 0,
  'Classes': 0,
  'Region': 0},
 'describe':                day       month    year  Temperature          RH          Ws  \
 count   243.000000  243.000000   243.0   243.000000  243.000000  243.000000   
 unique         N